# RGB Image Reconstruction VAE

Code used is currently heavily based on a tutorial getting started with variational autoencoder using pytorch on Debugger Cafe  
This will be changed over time

In [ ]:
import os
import csv
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
from tqdm import tqdm
#import argparse

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.utils import save_image

matplotlib.style.use('ggplot')

torch.cuda.empty_cache() 
#import model

In [ ]:
class ActiveVisionDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, index_col=None)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
            index = index.item()
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        shape_label = torch.tensor(int(self.annotations.iloc[index,1]))
        #cam_loc = torch.tensor(int(self.annotations.iloc[index,2]))
        
        if self.transform:
            image = self.transform(image)
        
        return image, shape_label

In [ ]:
class Encoder(nn.Module):
    def __init__(self, z_dim):
        super(Encoder, self).__init__() 
        self.enc1 = nn.Conv2d(
            in_channels=3, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc2 = nn.Conv2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc3 = nn.Conv2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc4 = nn.Conv2d(
            in_channels=init_kernel*4, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc5 = nn.Conv2d(
            in_channels=init_kernel*8, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        #self.dropout = nn.Dropout2d(p=0.1) # need to call in forward but turn off automatically in eval()
        
        self.mu = nn.Linear(init_kernel*191*191, z_dim)
        self.sigma = nn.Linear(init_kernel*191*191, z_dim)
        
        
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample
 
    def forward(self, x):
        
        #print("before anything")
        #print(x.shape)
        x = self.enc1(x)
        x = F.relu(x)
        x = self.enc2(x)
        x = F.relu(x)
        x = self.enc3(x)
        x = F.relu(x)
        #x = self.enc4(x)
        #x = F.relu(x)
        #x = self.enc5(x)
        #x = F.relu(x)
        
        #print("before flatten:")
        #print(x.shape)
        
        x = x.view(x.size(0), -1)
        
        #print("after flatten:")
        #print(x.shape)
        
        # get `mu` and `log_var`
        mu = self.mu(x)
        log_var = self.sigma(x) # fix!!!!name
        
        #return mu, log_var #fix!!! name
        ## get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        
        return z

In [ ]:
class Decoder(nn.Module):
    def __init__(self, z_dim):
        super(Decoder, self).__init__()
        
        self.lin1 = nn.Linear(z_dim, init_kernel*191*191)
        
        self.dec1 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_kernel*8, out_channels=init_kernel*4, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec5 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=3, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        
    def forward(self, z):
        x = self.lin1(z)
        x=F.relu(x)
        
        x=x.view(-1, init_kernel, 191, 191)
        #print("after unflatten:")
        #print(x.shape)
        
        #x = self.dec1(x)
        #x = F.relu(x)
        #x = self.dec2(x)
        #x = F.relu(x)
        x = self.dec3(x)
        x = F.relu(x)
        x = self.dec4(x)
        x = F.relu(x)
        x = self.dec5(x)
        reconstruction = torch.sigmoid(x)
        
        return reconstruction

In [ ]:
class ConvVAE(nn.Module):
    def __init__(self, z_dim):
        super(ConvVAE, self).__init__()
        self.encoder = Encoder(z_dim)
        self.decoder = Decoder(z_dim)
    
    def forward(self,x):
        sample = self.encoder(x)
        #z_mu, z_logvar = self.encoder(x)
        
        #std = torch.exp(z_logvar)
        #eps = torch.randn_like(std) # `randn_like` as we need the same size
        #sample = z_mu + (eps * std) # sampling
        
        reconstruction = self.decoder(sample)
        
        return reconstruction, z_mu, z_logvar

In [ ]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        #print(data)
        data, _ = data
        if torch.cuda.is_available():
            data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [ ]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, _ = data
            if torch.cuda.is_available():
                data = data.to(device)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(val_data)/dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data.view(batch_size, 3, 200, 200)[:8], 
                                  reconstruction.view(batch_size, 3, 200, 200)[:8]))
                save_image(both.cpu(), f"outputs/{parameter}{value}/imgs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

## Define Parameters (change in optimization)

In [ ]:
# leanring parameters

#Number of conv layers: 3,4,5

epochs = 50 # 50,100
batch_size = 8 #8,16,32,64?
lr = 0.0001 #0.002, 0.003

kernel_size = 4 #3,4,5,6?
stride = 1 #1,2,3,4?
padding = 0 #0,1,2,3,4?
init_kernel = 8 #4,8,16,32 #initial number of filters

latent_dim = 96

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Add remove layers ***make priority!!!!
#Optimizer (setup just a few cells below)

#Do dropout (across multiple layers, with multiple p values?)
#Do pooling? Max pool or average pool?
#Do activation function #There's a lot
#Batch normalization #Looked it up

## Run

In [ ]:
dataset = ActiveVisionDataset(csv_file='imgs/rgbCSV.csv', root_dir= 'imgs/rgbImg/', transform = torchvision.transforms.ToTensor())
train_data, val_data = torch.utils.data.random_split(dataset, [2400, 600])
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)

In [ ]:
dbsjhf

In [ ]:
batch_sizes = [8]

for batch_size in batch_sizes:
    
    dataset = ActiveVisionDataset(csv_file='imgs/rgbCSV.csv', root_dir= 'imgs/rgbImg/', transform = torchvision.transforms.ToTensor())
    train_data, val_data = torch.utils.data.random_split(dataset, [2400, 600])
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
    
    model = ConvVAE(latent_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss(reduction='sum')

    parameter = 'batch_sizes'
    value = batch_size
    os.makedirs("outputs/"+parameter+str(value), exist_ok=True)
    os.makedirs("outputs/"+parameter+str(value)+"/imgs", exist_ok=True)
    #parameters = ['layers']
    #values = ['3,4,5,6,7']

    #for parameter in parameters:
        #for value in values:

            #Create a folder here
            #os.makedirs(parameter+value, exist_ok=True)
    value = str(value)
    train_loss = []
    val_loss = []
    for epoch in range(epochs):
        print(f"Epoch {epoch+1} of {epochs}")
        train_epoch_loss = fit(model, train_loader)
        val_epoch_loss = validate(model, val_loader)
        train_loss.append(train_epoch_loss)
        val_loss.append(val_epoch_loss)
        print(f"Train Loss: {train_epoch_loss:.4f}")
        print(f"Val Loss: {val_epoch_loss:.4f}")

    filepath = os.path.join(os.getcwd(), "outputs", parameter+str(value), parameter+str(value)+".pth")
    torch.save(model.state_dict(), filepath)
    
    plt.figure(figsize=(10,10))
    plt.plot(range(1,epochs+1), train_loss, label="Train Loss")
    plt.plot(range(1,epochs+1), val_loss, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.yticks(np.arange(int(math.floor(min(train_loss) / 100.0)) * 100, max(train_loss)+1, 1000))
    plt.savefig('outputs/'+parameter+value+'/loss'+parameter+value+'.png')
    
    with open('outputs/'+parameter+value+'/loss'+parameter+value+'.csv','w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(["Train loss", "Val loss"])
        wr.writerows(zip(train_loss, val_loss))

    with open('outputs/lossCompare.csv', 'a+', newline='') as f:
        wr = csv.writer(f)
        wr.writerow([parameter, value ,train_loss[-1], val_loss[-1]])

In [ ]:
ok boomer

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(range(1,epochs+1), train_loss, label="Train Loss")
plt.plot(range(1,epochs+1), val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.yticks(np.arange(int(math.floor(min(train_loss) / 100.0)) * 100, max(train_loss)+1, 1000))
plt.savefig('outputs/'+parameter+value+'/loss'+parameter+value+'.png')

In [ ]:
with open('outputs/'+parameter+value+'/loss'+parameter+value+'.csv','w', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["Train loss", "Val loss"])
    wr.writerows(zip(train_loss, val_loss))

with open('outputs/lossCompare.csv', 'a+', newline='') as f:
    wr = csv.writer(f)
    wr.writerow([parameter, value ,train_loss[-1], val_loss[-1]])

### C1ASSIFICATIONNN

In [ ]:
classes = ['Shape 0', 'Shape 1', 'Shape 2', 'Shape 3', 'Shape 4', 'Shape 5']

In [ ]:

classes = ['Shape 0', 'Shape 1', 'Shape 2', 'Shape 3', 'Shape 4', 'Shape 5']

# functions to show an image


def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

'''
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(8)))
'''

In [ ]:
#Which model am i tryna load in huh
PATH = os.path.join(os.getcwd(), "outputs", "batch_sizes8", "batch_sizes8.pth")

ConvVAE = ConvVAE(latent_dim)#.to(device)
ConvVAE.load_state_dict(torch.load(PATH))

In [ ]:
ConvVAE.eval()

In [ ]:
#Should probably be a test set here!

def generate_latent_vectors(model, dataloader):
    model.eval()
    latent = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, _ = data
            #if torch.cuda.is_available():
            #    data = data.to(device)
            z = ConvVAE.encoder(data)
            latent.append(z)
        return latent

In [ ]:
train_latent_var = generate_latent_vectors(ConvVAE, train_loader)
test_latent_var = generate_latent_vectors(ConvVAE, val_loader)

In [ ]:
'''
print(len(latent_var))
train_latent_var = latent_var[0:60]
test_latent_var = latent_var[60:75]
print(len(train_latent_var))
print(len(test_latent_var))
'''

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(3, 6, 5)
        #self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(96, 60)
        #self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(60, 30)

    def forward(self, x):
        #x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        _, labels = data
        z = train_latent_var[i]
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        #if torch.cuda.is_available():
        #    z = z.to(device)

        # forward + backward + optimize
        outputs = net(z)#.to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

In [ ]:
dataiter = iter(val_loader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(8)))

In [ ]:
for i in range(0,8):
    outputs = net(test_latent_var[i])

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(8)))

In [ ]:
correct = 0
total = 0
prediction = []
actual = []
with torch.no_grad():
    i = 0
    for data in val_loader:
        images, labels = data
        outputs = net(test_latent_var[i])
        _, predicted = torch.max(outputs.data, 1)
        prediction.append(predicted.tolist())
        actual.append(labels.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        i+=1
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
print("prediction")
print(prediction)

print("actual")
print(actual)

## Extra helper code

1. Adding/removing layers: CNN layers and fully connected layers  
2. Varying batch sizes  
3. Varying numbers of training epochs  
4. Applying different levels of drop-out  
5. Applying different types of pooling  
6. Applying different types of activation function  

In [ ]:
#for data in train_loader:
#    print(im_path)
#    print("Data: ", data)

In [ ]:
#print(list(train_data))
#type(train_loader)
#print(train_data.shape)

for i, (images, labels) in enumerate(train_loader):
    print(images.shape)
#    print(labels.shape)
#    print(labels)

In [ ]:
#print(model)

In [ ]:
for i in range(0,10):
    vars()['bruh'+str(i)] = i

print (bruh0)

In [ ]:
        for i in range(2,6):
            vars()[self.'enc'+str(i)] = nn.Conv2d(
                in_channels=init_kernel, out_channels=init_kernel*i, kernel_size=kernel_size, 
                stride=stride, padding=padding
            )

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = ConvVAE.encoder(images)
        _, predicted = torch.max(outputs.data, 1)
        print(outputs)
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))